In [1]:
import pandas as pd
import sqlite3
import numpy as np

In [2]:
import sys
sys.path.append("../module/")
from lipinski import *

In [14]:
from rdkit import Chem
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
import pandas as pd
import sqlite3
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing

from matplotlib import pyplot as ptl
import seaborn as sns
import sys, os
from rdkit import Chem
from rdkit.Chem import Crippen
from rdkit.Chem import Lipinski
from rdkit.Chem import Descriptors
from rdkit.Chem.QED import properties

import subprocess
import threading
import queue


In [3]:
con = sqlite3.connect(f"../data/propriedades.db")
dataframe = pd.read_sql('select * from dados', con)
dataframe

,chembl_id,mw_freebase,alogp,psa,rtb,cx_most_apka,cx_most_bpka,cx_logp,cx_logd,molecular_species,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,hba_lipinski,hbd_lipinski,num_lipinski_ro5_violations,canonical_smiles
0,CHEMBL287510,357.454,1.5119,73.83,10.0,NaN,8.79,1.80,0.40,BASE,357.45,2.0,26.0,0.56,357.2052,5.0,3.0,0.0,CN(C)c1ccccc1C(=O)NCCNCC(O)COc1ccccc1
1,CHEMBL285301,320.433,1.7124,70.59,9.0,NaN,8.79,1.96,0.55,BASE,320.43,2.0,23.0,0.61,320.2100,4.0,3.0,0.0,O=C(NCCNCC(O)COc1ccccc1)C1CCCCC1
2,CHEMBL285413,255.314,1.4123,48.95,8.0,NaN,7.42,1.45,1.14,NEUTRAL,255.31,1.0,18.0,0.71,255.1471,5.0,1.0,0.0,COc1ccc(OC)c(CNCC(OC)OC)c1
3,CHEMBL24475,262.349,3.6489,46.53,8.0,9.95,NaN,4.40,4.40,NEUTRAL,262.35,1.0,19.0,0.57,262.1569,3.0,1.0,0.0,CCCC/C=C/C(=O)CCc1ccc(O)c(OC)c1
4,CHEMBL24545,330.468,4.4500,43.37,2.0,NaN,NaN,4.21,4.21,NEUTRAL,330.47,4.0,24.0,0.70,330.2195,3.0,0.0,0.0,CCC(=O)OC1CCC2C3CCC4=CC(=O)CCC4C3CCC12C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084583,CHEMBL4110107,507.538,4.4727,105.57,8.0,11.72,3.32,3.13,3.13,NEUTRAL,507.54,4.0,35.0,0.47,507.1552,7.0,2.0,1.0,Cc1cc(N=S2(=O)CCC2)cc2ncnc(Nc3ccc(F)cc3O[C@H](...
2084584,CHEMBL4110108,337.335,3.0320,76.68,2.0,11.84,5.74,2.11,2.10,NEUTRAL,337.34,5.0,25.0,0.78,337.1063,5.0,1.0,0.0,O=C1OC[C@H](c2ccc3c(c2)OCCO3)N1c1ccc2nc[nH]c2c1
2084585,CHEMBL4110109,338.785,4.1153,41.99,7.0,NaN,7.09,3.66,3.49,NEUTRAL,338.79,2.0,23.0,0.77,338.0997,3.0,1.0,0.0,CC[C@@H](NCCF)c1ccc(Cl)c(C(=O)c2cccnc2)c1F
2084586,CHEMBL4110110,480.968,3.1266,96.81,10.0,NaN,3.97,3.19,3.19,NEUTRAL,480.97,3.0,33.0,0.54,480.1940,7.0,2.0,0.0,CC(C(=O)NCCF)c1ccc(O[C@@H]2CCN(c3ncnc(OCC(C)(C...


In [11]:
teste = dataframe.loc[dataframe.canonical_smiles.str.len() < 128 and dataframe.canonical_smiles.str.contains()]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [5]:
x = teste.to_numpy().tolist()

['CN(C)c1ccccc1C(=O)NCCNCC(O)COc1ccccc1',
 'O=C(NCCNCC(O)COc1ccccc1)C1CCCCC1',
 'COc1ccc(OC)c(CNCC(OC)OC)c1',
 'CCCC/C=C/C(=O)CCc1ccc(O)c(OC)c1',
 'CCC(=O)OC1CCC2C3CCC4=CC(=O)CCC4C3CCC12C',
 'C#CCN(Cc1ccc2nc(C(F)(F)F)nc(O)c2c1)c1ccc(C(=O)NC(CCC(=O)O)C(=O)O)cc1',
 'CCCCCCCOS(N)(=O)=O',
 'CCCCCCCCOS(N)(=O)=O',
 'CCCCCCCCCOS(N)(=O)=O',
 'COc1c2ccoc2cc2oc(=O)ccc12',
 'NC(=O)Cc1c(C(=O)O)[nH]c2cc(Cl)ccc12',
 'CCCOC(=O)Cc1c(C(=O)O)[nH]c2cc(Cl)ccc12',
 'CC(=O)NCCNCC(O)COc1ccccc1',
 'CC(CNC(=O)C(C)C)NCC(O)COc1ccccc1',
 'CC(C)C(=O)NCCNCC(O)COc1ccccc1[N+](=O)[O-]',
 'CCCCCC(=O)NCCNCC(O)COc1ccccc1',
 'CC(=O)NC[C@@H](Cc1ccccc1)N(C)C(=O)[C@@H](Cc1ccc2ccccc2c1)N(C)C(=O)/C=C/CC(C)(C)N',
 'CN(C(=O)[C@@H](Cc1ccc2ccccc2c1)N(C)C(=O)/C=C/CC(C)(C)N)[C@@H](CNC(=O)CCCO)Cc1ccccc1',
 'COc1cc(CCC(C)=O)ccc1O',
 'CCCCCC/C=C/C(=O)CCc1ccc(O)c(OC)c1',
 'N=C(N)c1ccc(-c2ccc(-c3ccc(C(=N)N)cc3)o2)cc1',
 'N=C(NC1CCCC1)c1ccc(-c2ccc(-c3ccc(C(=N)NC4CCCC4)cc3)o2)cc1',
 'N=C(NC1CCC1)c1ccc(-c2ccc(-c3ccc(C(=N)NC4CCC4)cc3)o2)cc1'

In [6]:
def ver_lip(x, i):
    file = open(f"smiles_erro_log_{i}.log",'a')
    for smiles in x:
        try:
            verifica_lipinski(smiles)
        except:
            file.write(f"{smiles}\n")
    file.close()

In [7]:
def verifica_kuku(array, threads_num):
  jobs = np.array_split(array, threads_num)
  processos = []    
  for i in range(1, threads_num):
    processThread = threading.Thread(target=ver_lip, args=(jobs.pop(), i))
    processThread.start()
    processos.append(processThread)
  ver_lip(jobs.pop(), 0)
  for proc in processos:
    proc.join()